In [11]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [12]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [13]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [14]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

#         if len(values)>0:  
#             BranchedMatrix.append([branched,values])
# #             BranchedValue.append(values)
# #             print 'BranchedMatrix',BranchedMatrix

#             if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
#                 SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
#                 SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
#                 print 'SecondModelOutput[1]',SecondModelOutput[1]
#                 print 'SecondModelOutput[0]',SecondModelOutput[0]
#                 if SecondModelOutput[1]>0.000001:
#     #                 print 'bayad cut bezane'
#                     output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
#                     print 'output',output[0]
#                     print 'output[1]',output[1]
#                     lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
#                     print 'lhs',lhs[0]
#                     self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
#                     print 'cut added'



In [15]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

admipex3(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Strategy_Search                     1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 25 rows, 25 columns, and 621 nonzeros.
Reduced MIP has 25 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.17 ticks)
Probing time = 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
Reduced MIP has 25 rows, 25 columns, and 621 nonzeros.
Reduced MIP has 25 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.24 ticks)
Probing time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.14 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  

     10    11      761.5741     6      730.0000      825.4669      141   13.08%             x24 U     10      9      8
Elapsed time = 0.42 sec. (12.96 ticks, tree = 0.01 MB, solutions = 3)
self.times_called 37
get_num_nodes 11
values [1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x2', 'x3', 'x5', 'x6', 'x8', 'x9', 'x10', 'x11', 'x12', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24']
branched [2, 3, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]
self.times_called 38
get_num_nodes 11
values [1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x2', 'x3', 'x5', 'x6', 'x8', 'x9', 'x10', 'x11', 'x12', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24']
branched [2, 3, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]
     11    10      761.5741     6      730.0000      825.4669      14

     27    21      805.0225    10      730.0000      818.9965      212   12.19%             x17 U     27     26      3
self.times_called 69
get_num_nodes 28
values [1.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x10', 'x16', 'x17', 'x20', 'x24']
branched [0, 10, 16, 17, 20, 24]
self.times_called 70
get_num_nodes 28
values [1.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x10', 'x16', 'x17', 'x20', 'x24']
branched [0, 10, 16, 17, 20, 24]
     28    22      797.5090     8      730.0000      818.9965      218   12.19%             x24 U     28     27      4
self.times_called 71
get_num_nodes 29
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x5', 'x10', 'x16', 'x17', 'x19', 'x20', 'x24']
branched [0, 5, 10, 16, 17, 19, 20, 24]
self.times_called 72
get_num_nodes 29
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x5', 'x10', 'x16', 'x17', 'x19', 'x20', 'x24']
branched [0, 5, 10, 16, 17, 19, 20, 24]
     29    23      791.8218     8     

     42    27      760.6752     7      730.0000      818.6148      275   12.14%             x15 D     42     41      6
self.times_called 95
get_num_nodes 43
values [-0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x3', 'x4', 'x5', 'x6', 'x9', 'x10', 'x11', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24']
branched [0, 1, 3, 4, 5, 6, 9, 10, 11, 15, 16, 17, 18, 19, 20, 21, 23, 24]
self.times_called 96
get_num_nodes 43
values [-0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x3', 'x4', 'x5', 'x6', 'x9', 'x10', 'x11', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24']
branched [0, 1, 3, 4, 5, 6, 9, 10, 11, 15, 16, 17, 18, 19, 20, 21, 23, 24]
     43    28      756.9602     5      730.0000      818.6148      277   12.14%              x6 U     43     42      7
self.times_called 97
get_num_nodes 44
values [-0.0, 1.0, -

values [-0.0, -0.0, -0.0]
branchedNames ['x10', 'x14', 'x15']
branched [10, 14, 15]
     59    33      796.3897     7      730.0000      807.6595      338   10.64%             x14 N     59      3      3
self.times_called 121
get_num_nodes 60
values [-0.0, -0.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x10', 'x14', 'x15', 'x17']
branched [0, 10, 14, 15, 17]
self.times_called 122
get_num_nodes 60
values [-0.0, -0.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x10', 'x14', 'x15', 'x17']
branched [0, 10, 14, 15, 17]
     60    34      782.3764     5      730.0000      807.6595      344   10.64%              x0 D     60     59      4
Elapsed time = 1.14 sec. (16.12 ticks, tree = 0.01 MB, solutions = 3)
self.times_called 123
get_num_nodes 61
values [-0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x0', 'x5', 'x10', 'x11', 'x12', 'x14', 'x15', 'x17', 'x19', 'x21', 'x24']
branched [0, 5, 10, 11, 12, 14, 15, 17, 19, 21, 24]
self.times_called 124
get_num_nodes 61
values [-0.

...

values [1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x4', 'x5', 'x6', 'x9', 'x10', 'x11', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22']
branched [0, 1, 4, 5, 6, 9, 10, 11, 15, 17, 18, 19, 20, 21, 22]
    177    48      751.3976     7      730.0000      776.5108      876    6.37%              x4 D    177    176      7
self.times_called 292
get_num_nodes 178
values [1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
branchedNames ['x0', 'x1', 'x4', 'x5', 'x6', 'x9', 'x10', 'x11', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23']
branched [0, 1, 4, 5, 6, 9, 10, 11, 15, 17, 18, 19, 20, 21, 22, 23]
self.times_called 293
get_num_nodes 178
values [1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
branchedNames ['x0', 'x1', 'x4', 'x5', 'x6', 'x9', 'x10', 'x11', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23']
branched [0, 1, 4, 5, 6, 9, 10, 11, 1

    192    44      752.5804     8      730.0000      772.5973      941    5.84%             x23 U    192    119      7
self.times_called 313
get_num_nodes 193
values [1.0, -0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x3', 'x4', 'x5', 'x9', 'x10', 'x12', 'x14', 'x15', 'x16', 'x17', 'x19', 'x20', 'x22', 'x23']
branched [0, 3, 4, 5, 9, 10, 12, 14, 15, 16, 17, 19, 20, 22, 23]
self.times_called 314
get_num_nodes 193
values [1.0, -0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x3', 'x4', 'x5', 'x9', 'x10', 'x12', 'x14', 'x15', 'x16', 'x17', 'x19', 'x20', 'x22', 'x23']
branched [0, 3, 4, 5, 9, 10, 12, 14, 15, 16, 17, 19, 20, 22, 23]
    193    45      752.5793     8      730.0000      772.5973      942    5.84%              x4 D    193    192      8
    194    44        cutoff            730.0000      772.5973      945    5.84%              x1 U    194    193      9
    195    43        cu

values [1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x6', 'x8', 'x10', 'x11', 'x12', 'x14', 'x15', 'x17', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24']
branched [1, 6, 8, 10, 11, 12, 14, 15, 17, 19, 20, 21, 22, 23, 24]
    212    40      758.2478     6      730.0000      767.2693     1044    5.11%              x6 N    212     69      6
self.times_called 335
get_num_nodes 213
values [1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x4', 'x6', 'x8', 'x10', 'x11', 'x12', 'x14', 'x15', 'x17', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24']
branched [1, 4, 6, 8, 10, 11, 12, 14, 15, 17, 19, 20, 21, 22, 23, 24]
self.times_called 336
get_num_nodes 213
values [1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x4', 'x6', 'x8', 'x10', 'x11', 'x12', 'x14', 'x15', 'x17', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24']
branched [1, 4, 6, 

values [-0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0]
branchedNames ['x1', 'x5', 'x6', 'x10', 'x12', 'x15', 'x17', 'x18', 'x19', 'x20', 'x24']
branched [1, 5, 6, 10, 12, 15, 17, 18, 19, 20, 24]
    227    37      741.0325     7      730.0000      760.0938     1091    4.12%             x24 D    227     80      6
    228    36        cutoff            730.0000      760.0938     1094    4.12%             x22 D    228    227      7
    229    35        cutoff            730.0000      760.0938     1097    4.12%             x22 U    229    227      7
self.times_called 355
get_num_nodes 230
values [-0.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0]
branchedNames ['x5', 'x6', 'x10', 'x11', 'x15', 'x19', 'x23']
branched [5, 6, 10, 11, 15, 19, 23]
self.times_called 356
get_num_nodes 230
values [-0.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0]
branchedNames ['x5', 'x6', 'x10', 'x11', 'x15', 'x19', 'x23']
branched [5, 6, 10, 11, 15, 19, 23]
self.times_called 357
get_num_nodes 230
values [-0.0, 1.0, -0.0,

    267     2        cutoff            730.0000      732.1510     1191    0.29%             x23 U    267    150      5
    268     1    infeasible            730.0000      731.8426     1191    0.25%             x22 U    268    109     11
    269     0        cutoff            730.0000      730.0000     1193    0.00%             x18 D    269    170      8

Cover cuts applied:  48
Mixed integer rounding cuts applied:  4
Zero-half cuts applied:  6

Root node processing (before b&c):
  Real time             =    0.27 sec. (11.79 ticks)
Sequential b&c:
  Real time             =    3.31 sec. (25.93 ticks)
                          ------------
Total (root+branch&cut) =    3.58 sec. (37.72 ticks)
MIP_optimal
Objective value =  730.0

Column 0: Value =                 1
Column 1: Value =                 1
Column 2: Value =                 1
Column 5: Value =                 1
Column 9: Value =                 1
Column 11: Value =                 1
Column 14: Value =                 1
Column 17